### Step 1 - Start Spark Session

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType
from pyspark.sql.functions import current_timestamp

# Initialize a Spark session
spark = SparkSession.builder.appName("PitStopsIngestion").getOrCreate()

23/12/28 14:32:01 WARN Utils: Your hostname, falcao-sys resolves to a loopback address: 127.0.1.1; using 192.168.11.185 instead (on interface wlx7898e8c12476)
23/12/28 14:32:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/28 14:32:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Step 2 - Read the JSON file sing the Spark Dataframe Reader

In [3]:
pit_stops_schema = StructType(fields=[
    StructField("raceId", IntegerType(), False),
    StructField("driverId", IntegerType(), False),
    StructField("stop", StringType(), False),
    StructField("lap", StringType(), False),
    StructField("time", StringType(), True),
    StructField("duration", StringType(), False),
    StructField("milliseconds", IntegerType(), True)
])

In [4]:
pit_stops_df = spark.read\
    .schema(pit_stops_schema)\
    .option("multiLine", True)\
    .json("../data/pit_stops.json")

### Step 3 - Rename columns and add new columns

In [ ]:
pit_stops_df = pit_stops_df\
    .withColumnRenamed("raceId", "race_id")\
    .withColumnRenamed("driverId", "driver_id")\
    .withColumn("ingestion_date", current_timestamp())

### Step 4 - Write to output to process container in Parquet Format

In [6]:
pit_stops_df.write.mode("overwrite").parquet("../processed_data/pit_stops")